In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
#from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#from sklearn.model_selection import GridSearchCV
#from sklearn.kernel_ridge import KernelRidge
# 주석 처리는 non-linear regression model인 svr에 관련된 import

In [7]:
store = pd.read_csv("data/commercial_with_xy.csv")
store.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,집객시설_수,아파트_평균_면적,아파트_평균_시가,상권_변화_지표,...,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,구분,점포_수,매출_금액,매출_건수,엑스좌표_값,와이좌표_값
0,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,개인_서비스업,116,3.230693e+07,398,196193,456641
1,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,교육_및_사업자원_서비스업,128,2.660860e+07,166,196193,456641
2,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,기타_도소매업,98,2.564806e+06,96,196193,456641
3,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,생활용품_도소매업,151,3.070350e+07,358,196193,456641
4,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,수리업,24,1.999051e+08,874,196193,456641


In [8]:
store.columns

Index(['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '총_생활인구_수',
       '집객시설_수', '아파트_평균_면적', '아파트_평균_시가', '상권_변화_지표', '상권_변화_지표_명',
       '운영_영업_개월_평균', '폐업_영업_개월_평균', '서울_운영_영업_개월_평균', '서울_폐업_영업_개월_평균', '구분',
       '점포_수', '매출_금액', '매출_건수', '엑스좌표_값', '와이좌표_값'],
      dtype='object')